# Weather Classification Pipeline using Tfx

## Library Import

**Penjelasan tahapan** : Tahapan ini dilakukan untuk mengambil library yang dibutuhkan untuk menjalankan semua code yang terdapat dalam file notebook (.ipynb) ini. Penjelasan mengenai fungsi masing-masing library yang di-import dapat dibaca pada baris code.

In [1]:
import os, random # Mengimport Library OS untuk Berinteraksi Dengan Sistem Operasi dan Random untuk Menggunakan Fungsi Acak
from pathlib import Path # Mengimport Modul Path Dari Library pathlib untuk Menangani Path File Dengan Gaya Object-Oriented
import json # Mengimport Library JSON untuk Bekerja Dengan Data JSON
from tfx import v1 as tfx # Mengimport Library TensorFlow Extended (TFX) dan Memberi Alias v1 sebagai tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext # Mengimport InteractiveContext untuk Menjalankan Komponen TFX Secara Interaktif
from tfx.components import ImportExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Tuner, Trainer, Evaluator, Pusher # Mengimport Komponen-Komponen Utama TFX yang Digunakan Dalam Pipeline
from tfx.proto import example_gen_pb2, pusher_pb2 # Mengimport Proto Messages untuk Mengkonfigurasi Komponen ExampleGen dan Pusher
from tfx.dsl.components.common.resolver import Resolver # Mengimport Resolver yang Berfungsi untuk Menyelesaikan Artefak (Misalnya, Menemukan Model Terbaru)
from tfx.types import Channel # Mengimport Channel untuk Menghubungkan Output Satu Komponen ke Input Komponen Lain
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy # Mengimport Strategi untuk Memilih Model Terbaru yang Disetujui (Blessed)
from tfx.types.standard_artifacts import Model, ModelBlessing # Mengimport Artefak Standar TFX untuk Representasi Model dan Status Persetujuannya
import tensorflow as tf # Mengimport Library TensorFlow untuk Operasi Pembelajaran Mesin
import tensorflow_model_analysis as tfma # Mengimport TensorFlow Model Analysis untuk Menganalisis dan Mengevaluasi Model

## Set Variable

**Penjelasan tahapan** : Tahapan ini digunakan untuk mendefinisikan variabel-variabel penting yang dibutuhkan sepanjang eksekusi pipeline. Variabel ini mencakup informasi dasar seperti nama pipeline, lokasi direktori penyimpanan artefak, metadata eksekusi, sumber data input, hingga path untuk modul Python yang berisi kode preprocessing, training, dan tuning. Selain itu, pada tahap ini juga dilakukan inisialisasi InteractiveContext agar pipeline dapat dijalankan secara interaktif di notebook. Direktori yang dibutuhkan untuk root pipeline dan penyimpanan model serving juga dibuat agar proses eksekusi dapat berjalan tanpa error.

In [2]:
PIPELINE_NAME = "pipeline" # Menentukan Nama Pipeline yang Digunakan

PIPELINE_ROOT = os.path.join("bravo_alfon", PIPELINE_NAME) # Mendefinisikan Direktori Root untuk Semua Artefak Pipeline, Menggunakan os.path.join untuk Membuat Path yang Kompatibel di Berbagai Sistem Operasi
METADATA_PATH = os.path.join(PIPELINE_ROOT, "metadata.sqlite") # Mendefinisikan Path File Metadata SQLite yang Menyimpan Informasi Status Eksekusi Pipeline

DATA_ROOT = os.path.join("data", "tfrecords") # Mendefinisikan Direktori Sumber Data Input yang Berisi File TFRecords

SERVING_MODEL_DIR = os.path.join("serving_model", PIPELINE_NAME) # Mendefinisikan Direktori Tempat Model yang Diekspor Akan Disimpan untuk Servis

TRANSFORM_MODULE_FILE = "module/weather_transform_module.py" # Mendefinisikan Path File Python yang Berisi Kode Preprocessing (Transform)
TRAINER_MODULE_FILE = "module/weather_trainer_module.py" # Mendefinisikan Path File Python yang Berisi Kode Pelatihan Model (Trainer)
TUNER_MODULE_FILE = "module/weather_tuner_module.py" # Mendefinisikan Path File Python yang Berisi Kode Penyetelan Hyperparameter (Tuner)

context = InteractiveContext(pipeline_root=PIPELINE_ROOT) # Menginisialisasi Konteks Interaktif TFX, Memungkinkan Komponen Dijalankan Secara Satu Per Satu di Lingkungan Notebook

os.makedirs(PIPELINE_ROOT, exist_ok=True) # Membuat Direktori Root Pipeline Jika Belum Ada (exist_ok=True Mencegah Error Jika Direktori Sudah Ada)
os.makedirs(SERVING_MODEL_DIR, exist_ok=True) # Membuat Direktori Penyimpanan Model Servis Jika Belum Ada (exist_ok=True Mencegah Error Jika Direktori Sudah Ada)

## Dataset Conversion to TFRecord

**Penjelasan tahapan** : Tahapan ini digunakan untuk mengonversi dataset mentah berupa file gambar ke dalam format TFRecord, yaitu format biner yang dioptimalkan untuk digunakan oleh TensorFlow. Proses ini bertujuan agar data dapat dibaca lebih cepat dan efisien saat digunakan dalam training maupun evaluasi model.

Pada tahap ini dilakukan beberapa hal penting, yaitu:

1. Mendefinisikan direktori dan parameter konversi – lokasi data mentah, lokasi output TFRecord, rasio data train–eval, serta daftar nama kelas.

2. Membuat fungsi pembantu (_bytes_feature, _string_feature) untuk mengubah data gambar dan label menjadi format tf.train.Feature.

3. Menulis TFRecord menggunakan fungsi write_tfrecord, yang membuat objek tf.train.Example berisi fitur gambar dan label, lalu menyimpannya ke file TFRecord.

4. Membagi dataset menjadi train dan eval sesuai rasio yang ditentukan (80% untuk training, 20% untuk evaluasi), dengan pembagian acak berdasarkan seed agar hasil reproducible.

5. Menyimpan data hasil pembagian ke file TFRecord pada direktori train/data.tfrecord dan eval/data.tfrecord.

6. Menampilkan ringkasan hasil konversi berupa jumlah data per kelas di masing-masing subset (train dan eval).

Dengan cara ini, data gambar sudah siap dipakai oleh komponen ExampleGen di pipeline TFX untuk tahap selanjutnya.

In [3]:
RAW_DIR = "data/raw_images" # Mendefinisikan Direktori Sumber Gambar Mentah
OUT_DIR = DATA_ROOT # Mendefinisikan Direktori Output untuk Menyimpan File TFRecord
TRAIN_RATIO = 0.8 # Mendefinisikan Rasio Data Latihan (80%)
SEED = 42 # Mendefinisikan Seed untuk Reproducibility dalam Pengacakan
 
CLASS_NAMES = ["Cloudy", "Rain", "Shine", "Sunrise"] # Mendefinisikan Nama-Nama Kelas Kategori Cuaca
 
def _bytes_feature(v: bytes) -> tf.train.Feature: # Fungsi Pembantu untuk Mengubah Bytes Menjadi tf.train.Feature
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[v])) # Mengembalikan Objek Feature dengan BytesList dari Nilai yang Diberikan
 
def _string_feature(v: str) -> tf.train.Feature: # Fungsi Pembantu untuk Mengubah String Menjadi tf.train.Feature
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[v.encode("utf-8")])) # Mengembalikan Objek Feature dengan BytesList dari String yang Dienkode UTF-8
 
def write_tfrecord(records, out_path): # Fungsi Utama untuk Menulis Records ke File TFRecord
    os.makedirs(os.path.dirname(out_path), exist_ok=True) # Membuat Direktori untuk File Output Jika Belum Ada
    with tf.io.TFRecordWriter(out_path) as w: # Membuka TFRecordWriter untuk Menulis Data ke File
        for img_path, label_str in records: # Melakukan Iterasi untuk Setiap Pasangan Path Gambar dan Label
            with tf.io.gfile.GFile(img_path, "rb") as f: # Membuka File Gambar dalam Mode Binary Read
                img_bytes = f.read() # Membaca Seluruh Konten File Gambar sebagai Bytes
            example = tf.train.Example(features=tf.train.Features(feature={ # Membuat Objek tf.train.Example dari Fitur-Fitur Gambar dan Label
                "image": _bytes_feature(img_bytes), # Menambahkan Fitur 'image' dengan Data Bytes
                "label": _string_feature(label_str), # Menambahkan Fitur 'label' dengan Data String
            }))
            w.write(example.SerializeToString()) # Menulis Example yang Sudah Disesuaikan ke dalam File TFRecord
 
train_recs, eval_recs = [], [] # Menginisialisasi List Kosong untuk Records Latihan dan Evaluasi
rng = random.Random(SEED) # Menginisialisasi Generator Angka Acak dengan Seed yang Ditetapkan
 
for cls in CLASS_NAMES: # Melakukan Iterasi untuk Setiap Nama Kelas
    all_cls_imgs = [str(p) for p in Path(RAW_DIR, cls).rglob("*") if p.suffix.lower() in [".jpg",".jpeg",".png"]] # Mendapatkan Semua Path Gambar untuk Kelas Saat Ini
    rng.shuffle(all_cls_imgs) # Mengacak Urutan Daftar Gambar
    
    n_train = int(len(all_cls_imgs) * TRAIN_RATIO) # Menghitung Jumlah Gambar untuk Set Latihan
    train_cls = [(p, cls) for p in all_cls_imgs[:n_train]] # Membuat List Tuple (path, label) untuk Set Latihan
    eval_cls  = [(p, cls) for p in all_cls_imgs[n_train:]] # Membuat List Tuple (path, label) untuk Set Evaluasi
    
    train_recs.extend(train_cls) # Menambahkan Records Latihan Kelas Saat Ini ke Daftar Records Latihan Global
    eval_recs.extend(eval_cls) # Menambahkan Records Evaluasi Kelas Saat Ini ke Daftar Records Evaluasi Global
 
write_tfrecord(train_recs, os.path.join(OUT_DIR, "train", "data.tfrecord")) # Menulis Records Latihan ke File TFRecord 'train/data.tfrecord'
write_tfrecord(eval_recs,  os.path.join(OUT_DIR, "eval",  "data.tfrecord")) # Menulis Records Evaluasi ke File TFRecord 'eval/data.tfrecord'
 
print("TFRecord selesai.") # Mencetak Pesan Bahwa Proses TFRecord Telah Selesai
print("Train:", len(train_recs), "Eval:", len(eval_recs)) # Mencetak Jumlah Total Records di Set Latihan dan Evaluasi
for cls in CLASS_NAMES: # Melakukan Iterasi Lagi untuk Mencetak Detail Jumlah Data per Kelas
    n_tr = sum(1 for _, c in train_recs if c==cls) # Menghitung Jumlah Records untuk Kelas di Set Latihan
    n_ev = sum(1 for _, c in eval_recs  if c==cls) # Menghitung Jumlah Records untuk Kelas di Set Evaluasi
    print(f"{cls:8s} -> Train: {n_tr}, Eval: {n_ev}") # Mencetak Jumlah Records untuk Setiap Kelas di Kedua Set Data

TFRecord selesai.
Train: 899 Eval: 226
Cloudy   -> Train: 240, Eval: 60
Rain     -> Train: 172, Eval: 43
Shine    -> Train: 202, Eval: 51
Sunrise  -> Train: 285, Eval: 72


## ExampleGen

**Penjelasan tahapan** : Tahapan ini digunakan untuk memasukkan data ke dalam pipeline TFX agar bisa digunakan oleh komponen-komponen berikutnya. Pada tahap ini digunakan komponen ImportExampleGen yang berfungsi membaca data dalam format TFRecord yang sudah disiapkan sebelumnya. Konfigurasi input dilakukan dengan menentukan direktori dasar tempat data berada serta membagi data ke dalam dua subset, yaitu train dan eval, sesuai pola file yang ditentukan. Setelah konfigurasi selesai, komponen ExampleGen dijalankan menggunakan context.run(example_gen) sehingga data berhasil diimpor ke pipeline dan disimpan sebagai artefak yang dapat digunakan pada tahap validasi, transformasi, maupun pelatihan model.

In [4]:
DATA_ROOT = os.path.join("data", "tfrecords") # Mendefinisikan Direktori Utama Tempat Data TFRecord Disimpan
 
example_gen = ImportExampleGen( # Menginisialisasi Komponen ImportExampleGen untuk Mengimpor Data
    input_base=DATA_ROOT, # Menentukan Direktori Dasar (Root) untuk Data Input
    input_config=example_gen_pb2.Input(splits=[ # Mengonfigurasi Pembagian (Splitting) Data Input
        example_gen_pb2.Input.Split(name="train", pattern="train/*"), # Mendefinisikan Split 'train' dengan Mencari File yang Sesuai dengan Pola 'train/*'
        example_gen_pb2.Input.Split(name="eval", pattern="eval/*"), # Mendefinisikan Split 'eval' dengan Mencari File yang Sesuai dengan Pola 'eval/*'
    ])
)

In [5]:
context.run(example_gen) # Menjalankan Komponen ExampleGen dalam Konteks Interaktif

ExecutionResult(
    component_id: ImportExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=ImportExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

## StatisticGen

**Penjelasan tahapan** : Tahapan ini digunakan untuk menghasilkan statistik deskriptif dari data yang telah diimpor pada tahap ExampleGen. Komponen StatisticsGen membaca artefak keluaran berupa examples dari ExampleGen dan kemudian menghitung berbagai informasi statistik seperti jumlah sampel, distribusi label, nilai rata-rata, sebaran fitur, serta potensi nilai yang hilang. Statistik yang dihasilkan pada tahap ini akan sangat berguna untuk memahami karakteristik data sekaligus menjadi dasar bagi komponen berikutnya seperti SchemaGen dan ExampleValidator dalam mendeteksi anomali maupun masalah kualitas data.

In [6]:
statistics_gen = StatisticsGen( # Menginisialisasi Komponen StatisticsGen untuk Menghasilkan Statistik Deskriptif dari Data
    examples=example_gen.outputs['examples'] # Mendapatkan Output 'examples' dari Komponen example_gen Sebelumnya sebagai Input
)

In [7]:
context.run(statistics_gen) # Menjalankan Komponen StatisticsGen dalam Konteks Interaktif

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [8]:
context.show(statistics_gen.outputs["statistics"])

## SchemaGen

**Penjelasan tahapan** : Tahapan ini digunakan untuk menghasilkan skema data berdasarkan statistik yang telah dibuat pada tahap sebelumnya. Komponen SchemaGen membaca output statistik dari StatisticsGen kemudian secara otomatis menyimpulkan tipe data, domain nilai, dan bentuk (shape) fitur. Dengan opsi infer_feature_shape=True, SchemaGen juga akan mencoba mendeteksi struktur atau dimensi fitur dari data input. Skema yang dihasilkan berfungsi sebagai acuan standar yang merepresentasikan definisi data, sehingga dapat dipakai untuk memvalidasi konsistensi data di tahap ExampleValidator maupun sebagai pedoman saat preprocessing dan training model.

In [9]:
schema_gen = SchemaGen( # Menginisialisasi Komponen SchemaGen untuk Menghasilkan Skema Data
    statistics=statistics_gen.outputs['statistics'], # Menggunakan Output Statistik dari Komponen statistics_gen Sebagai Input
    infer_feature_shape=True # Menginstruksikan SchemaGen untuk Secara Otomatis Menyimpulkan Bentuk Fitur dari Statistik Input
)

In [10]:
context.run(schema_gen) # Menjalankan Komponen SchemaGen dalam Konteks Interaktif

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [11]:
context.show(schema_gen.outputs["schema"]) # Menampilkan Skema Data yang Dihasilkan

,Type,Presence,Valency,Domain
Feature name,,,,
'image',BYTES,required,,-
'label',STRING,required,,'label'


,Values
Domain,
'label',"'Cloudy', 'Rain', 'Shine', 'Sunrise'"


## ExampleValidator

**Penjelasan tahapan** : Tahapan ini digunakan untuk memvalidasi kualitas data dengan membandingkan statistik yang dihasilkan pada tahap StatisticsGen dengan skema yang telah didefinisikan pada tahap SchemaGen. Komponen ExampleValidator berfungsi untuk mendeteksi adanya anomali pada data seperti nilai yang hilang, data dengan tipe yang tidak sesuai, atau distribusi yang menyimpang dari skema yang diharapkan. Hasil dari tahap ini berupa laporan anomali yang dapat ditinjau kembali, sehingga permasalahan data bisa diidentifikasi sejak awal sebelum melanjutkan ke tahap transformasi dan pelatihan model.

In [12]:
example_validator = ExampleValidator( # Menginisialisasi Komponen ExampleValidator untuk Memvalidasi Data
    statistics=statistics_gen.outputs['statistics'], # Mengambil Output Statistik dari Komponen statistics_gen sebagai Input
    schema=schema_gen.outputs['schema'] # Mengambil Output Skema dari Komponen schema_gen sebagai Input
)

In [13]:
context.run(example_validator) # Menjalankan Komponen ExampleValidator dalam Konteks Interaktif

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [14]:
context.show(example_validator.outputs['anomalies']) # Menampilkan Hasil Anomali yang Ditemukan oleh ExampleValidator

## Transform

**Penjelasan tahapan** : Tahapan ini digunakan untuk melakukan pra-pemrosesan data sehingga siap dipakai pada tahap pelatihan model. Komponen Transform akan menjalankan fungsi preprocessing_fn yang telah dituliskan pada file Python terpisah. Dalam fungsi ini, data gambar yang masih berbentuk bytes akan didekode menjadi tensor, diubah ukurannya menjadi 128x128 piksel, serta dinormalisasi ke rentang nilai 0–1 agar lebih stabil saat proses training. Label yang semula berbentuk string juga dikonversi menjadi angka menggunakan tabel pencarian (lookup table). Hasil transformasi ini berupa fitur gambar dan label yang sudah terstandardisasi, sehingga pipeline dapat menggunakan data dengan format yang konsisten dan efisien untuk tahap training maupun evaluasi model.

In [15]:
%%writefile {TRANSFORM_MODULE_FILE} # Perintah Jupyter Notebook untuk Menulis Kode ke File Python
import tensorflow as tf # Mengimport Library TensorFlow untuk Melakukan Operasi Data dan Model
 
CLASS_NAMES = ["Cloudy", "Rain", "Shine", "Sunrise"] # Mendefinisikan Daftar Nama Kelas
IMG_SIZE = (128, 128) # Menentukan Ukuran Gambar yang Diinginkan
 
def preprocessing_fn(inputs): # Mendefinisikan Fungsi Pra-pemrosesan yang akan Dijalankan oleh Komponen Transform
    image_bytes = inputs['image'] # Mengambil Data Gambar dalam Format Bytes dari Input
    label_str = inputs['label'] # Mengambil Data Label dalam Format String dari Input
 
    def process_single_image(img_bytes): # Fungsi Pembantu untuk Memproses Satu Gambar Secara Independen
        img = tf.io.decode_image( # Mendekode Bytes Gambar Menjadi Tensor Gambar
            tf.reshape(img_bytes, []), # Memastikan Input Berbentuk Skalar (Scalar)
            channels=3, # Mengatur Gambar Agar Memiliki 3 Channel Warna (RGB)
            expand_animations=False # Memastikan Animasi (GIF) tidak Diperluas
        )
        img.set_shape([None, None, 3]) # Menetapkan Bentuk Tensor (Shape) Gambar
        img = tf.image.resize(img, IMG_SIZE) # Mengubah Ukuran Gambar menjadi IMG_SIZE
        img = tf.cast(img, tf.float32) / 255.0 # Mengubah Tipe Data Gambar Menjadi Float32 dan Menormalisasi Nilai Piksel ke Rentang 0-1
        return img # Mengembalikan Tensor Gambar yang Sudah Diproses
 
    images = tf.map_fn( # Menerapkan Fungsi process_single_image ke Seluruh Elemen di image_bytes
        process_single_image, # Fungsi yang akan Diterapkan pada Setiap Elemen
        image_bytes, # Tensor Input yang akan Dipetakan
        fn_output_signature=tf.TensorSpec(shape=(*IMG_SIZE, 3), dtype=tf.float32) # Mendefinisikan (Signature) Tensor Output
    )
 
    table = tf.lookup.StaticHashTable( # Membuat Hash Table untuk Mengubah Label String Menjadi Label Angka
        initializer=tf.lookup.KeyValueTensorInitializer( # Menginisialisasi Hash Table dengan Kunci dan Nilai
            keys=tf.constant(CLASS_NAMES), # Menggunakan Nama Kelas Sebagai Kunci
            values=tf.constant(list(range(len(CLASS_NAMES))), dtype=tf.int64), # Menggunakan Angka (Index) Sebagai Nilai
        ),
        default_value=-1 # Mengatur Nilai Default Jika Kunci Tidak Ditemukan
    )
    labels = table.lookup(label_str) # Mengubah Label String Menjadi Label Angka Menggunakan Hash Table
 
    return { # Mengembalikan Dictionary Berisi Fitur yang Telah Ditransformasi
        'image_xf': images, # Mengembalikan Tensor Gambar yang Telah Diproses
        'label_xf': labels # Mengembalikan Tensor Label yang Telah Ditransformasi Menjadi Angka
    }

Writing module/weather_transform_module.py


In [16]:
transform = Transform( # Menginisialisasi Komponen Transform
    examples=example_gen.outputs['examples'], # Mengambil Output 'examples' dari Komponen example_gen Sebagai Input
    schema=schema_gen.outputs['schema'], # Mengambil Output 'schema' dari Komponen schema_gen Sebagai Input
    module_file="module/weather_transform_module.py" # Menentukan File Modul Python yang Berisi Fungsi Pra-pemrosesan
)

In [17]:
context.run(transform) # Menjalankan Komponen Transform dalam Konteks Interaktif

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: bravo_alfon\pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\d5658540a2c142ebb52d2c767650d8fa\assets


INFO:tensorflow:Assets written to: bravo_alfon\pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\d5658540a2c142ebb52d2c767650d8fa\assets


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

## Tuner

**Penjelasan tahapan** : Tahapan ini digunakan untuk melakukan penyetelan hyperparameter model secara otomatis dengan memanfaatkan komponen Tuner dari TFX yang terintegrasi dengan KerasTuner. Pada tahap ini, data yang sudah melalui proses transformasi dimanfaatkan untuk membangun pipeline tuning, di mana model didefinisikan menggunakan beberapa parameter yang dapat diatur seperti jumlah filter pada layer konvolusi, jumlah unit pada dense layer, dropout rate, serta nilai learning rate. Proses tuning dilakukan menggunakan algoritma pencarian acak (random search) dengan tujuan menemukan kombinasi hyperparameter yang menghasilkan akurasi validasi terbaik. Dengan demikian, tahap Tuner membantu memilih konfigurasi model yang lebih optimal sebelum dilanjutkan ke tahap pelatihan penuh.

In [18]:
%%writefile {TUNER_MODULE_FILE} # Perintah Jupyter Notebook untuk Menulis Kode Berikut ke dalam File Python
import tensorflow as tf # Mengimpor TensorFlow untuk Membuat dan Melatih Model
import kerastuner as kt # Mengimpor KerasTuner untuk Penyetelan Hyperparameter
from tfx import v1 as tfx # Mengimpor TFX untuk Komponen dan Jenis Data
from tensorflow_transform import TFTransformOutput # Mengimpor TFTransformOutput untuk Mengakses Output Transform
import os # Mengimpor OS untuk Operasi Sistem File
from tfx.components.tuner.component import TunerFnResult # Mengimpor TunerFnResult untuk Mengembalikan Hasil dari tuner_fn
 
IMG_SIZE = (128, 128) # Menentukan Ukuran Standar untuk Semua Gambar Input
CLASS_NAMES = ["Cloudy", "Rain", "Shine", "Sunrise"] # Mendefinisikan Nama Kelas Kategori
 
def create_dataset(file_pattern, feature_spec, batch_size=32, shuffle=False): # Fungsi untuk Membuat tf.data.Dataset dari File TFRecord
    file_paths = tf.io.gfile.glob(file_pattern) # Mendapatkan Daftar Semua Path File yang Sesuai dengan Pola
    if not file_paths: # Memeriksa Apakah Ada File yang Ditemukan
        raise ValueError(f"No files found for pattern: {file_pattern}") # Mengangkat Kesalahan Jika Tidak Ada File yang Ditemukan
 
    dataset = tf.data.Dataset.from_tensor_slices(file_paths) # Membuat Dataset dari Daftar Path File
    dataset = dataset.interleave( # Memuat dan Memproses Data dari Berbagai File Secara Paralel
        lambda filepath: tf.data.TFRecordDataset(filepath, compression_type="GZIP"), # Fungsi untuk Membaca File TFRecord
        num_parallel_calls=tf.data.AUTOTUNE, # Menggunakan Panggilan Paralel Otomatis untuk Pemuatan Data yang Lebih Cepat
        cycle_length=min(4, len(file_paths)) # Menentukan Jumlah File yang akan Diproses Secara Bersamaan
    )
 
    def parse_record(serialized_example): # Fungsi Pembantu untuk Menguraikan Contoh Berseri dari TFRecord
        parsed_features = tf.io.parse_single_example(serialized_example, feature_spec) # Menguraikan Contoh ke Fitur-Fitur
        image = parsed_features['image_xf'] # Mengambil Fitur Gambar yang Telah Ditransformasi
        label = parsed_features['label_xf'] # Mengambil Fitur Label yang Telah Ditransformasi
        image = tf.ensure_shape(image, (*IMG_SIZE, 3)) # Memastikan Bentuk (Shape) Tensor Gambar
        label = tf.reshape(label, []) # Mengubah Bentuk Tensor Label menjadi Skalar (Scalar)
        return image, label # Mengembalikan Pasangan Gambar dan Label
 
    dataset = dataset.map(parse_record, num_parallel_calls=tf.data.AUTOTUNE) # Menerapkan Fungsi Parsing ke Setiap Rekor Secara Paralel
    if shuffle: # Memeriksa Apakah Data Perlu Diacak
        dataset = dataset.shuffle(buffer_size=1000) # Mengacak Dataset dengan Buffer yang Ditetapkan
 
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE) # Mengelompokkan Data dan Melakukan Prefetch untuk Performa
    return dataset # Mengembalikan Dataset yang Siap untuk Digunakan
 
def _build_keras_model(hp): # Fungsi untuk Membangun Model Keras dengan Hyperparameter yang Dapat Disetel
    model = tf.keras.Sequential([ # Membuat Model Sequential
        tf.keras.layers.InputLayer(input_shape=(*IMG_SIZE, 3), name="image_xf"), # Menambahkan Layer Input
 
        tf.keras.layers.Conv2D( # Layer Konvolusi Pertama
            filters=hp.Int("conv1_filters", 32, 64, step=16), # Menyetel Jumlah Filter
            kernel_size=(3, 3), activation="relu", padding="same"), # Menentukan Ukuran Kernel, Fungsi Aktivasi, dan Padding
        tf.keras.layers.MaxPooling2D((2, 2)), # Layer MaxPooling Pertama
 
        tf.keras.layers.Conv2D( # Layer Konvolusi Kedua
            filters=hp.Int("conv2_filters", 64, 128, step=32), # Menyetel Jumlah Filter
            kernel_size=(3, 3), activation="relu", padding="same"), # Menentukan Ukuran Kernel, Fungsi Aktivasi, dan Padding
        tf.keras.layers.MaxPooling2D((2, 2)), # Layer MaxPooling Kedua
 
        tf.keras.layers.Conv2D( # Layer Konvolusi Ketiga
            filters=hp.Int("conv3_filters", 128, 256, step=64), # Menyetel Jumlah Filter
            kernel_size=(3, 3), activation="relu", padding="same"), # Menentukan Ukuran Kernel, Fungsi Aktivasi, dan Padding
        tf.keras.layers.MaxPooling2D((2, 2)), # Layer MaxPooling Ketiga
 
        tf.keras.layers.Flatten(), # Meratakan Output dari Layer Sebelumnya
 
        tf.keras.layers.Dense( # Layer Dense Pertama
            units=hp.Int("dense_units", 64, 256, step=64), # Menyetel Jumlah Unit
            activation="relu"), # Menentukan Fungsi Aktivasi
        tf.keras.layers.Dropout(hp.Float("dropout_rate", 0.3, 0.5, step=0.1)), # Layer Dropout untuk Regularisasi
 
        tf.keras.layers.Dense(len(CLASS_NAMES), activation="softmax"), # Layer Output dengan Jumlah Unit Sama dengan Jumlah Kelas
    ])
 
    lr = hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4]) # Menyetel Learning Rate
    model.compile( # Mengompilasi Model
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr), # Menggunakan Optimizer Adam dengan Learning Rate yang Dapat Disetel
        loss="sparse_categorical_crossentropy", # Menentukan Fungsi Loss untuk Klasifikasi
        metrics=["accuracy"] # Menentukan Metrik untuk Evaluasi
    )
    return model # Mengembalikan Model yang Telah Dikompilasi
 
def tuner_fn(fn_args: tfx.components.FnArgs): # Fungsi Penyetelan Utama yang Dijalankan oleh Komponen Tuner
    print(">>> Debug: Starting tuner_fn") # Mencetak Pesan Debug
    
    if not hasattr(fn_args, 'custom_config') or not fn_args.custom_config: # Memeriksa Keberadaan Konfigurasi Kustom
        raise ValueError("custom_config is required for accessing transform graph") # Mengangkat Kesalahan Jika Konfigurasi Kustom Tidak Ada
    
    transform_graph_uri = fn_args.custom_config.get('transform_graph_uri') # Mendapatkan URI dari Grafik Transform
    if not transform_graph_uri: # Memeriksa Keberadaan URI
        raise ValueError("transform_graph_uri not found in custom_config") # Mengangkat Kesalahan Jika URI Tidak Ditemukan
    
    print(f"Using transform graph from: {transform_graph_uri}") # Mencetak URI Grafik Transform
    
    if not tf.io.gfile.exists(transform_graph_uri): # Memeriksa Apakah Path Grafik Transform Benar-Benar Ada
        raise ValueError(f"Transform graph path does not exist: {transform_graph_uri}") # Mengangkat Kesalahan Jika Path Tidak Ditemukan
    
    tf_transform_output = TFTransformOutput(transform_graph_uri) # Memuat Output Transform dari URI
    feature_spec = tf_transform_output.transformed_feature_spec() # Mendapatkan Spesifikasi Fitur yang Telah Ditransformasi
    print("Feature spec loaded successfully") # Mencetak Pesan Berhasil
 
    train_dataset = create_dataset(fn_args.train_files, feature_spec, batch_size=32, shuffle=True) # Membuat Dataset Latihan
    eval_dataset = create_dataset(fn_args.eval_files, feature_spec, batch_size=32, shuffle=False) # Membuat Dataset Evaluasi
 
    train_dataset = train_dataset.repeat() # Mengatur Dataset Latihan agar Berulang Tanpa Batas
    eval_dataset = eval_dataset.repeat() # Mengatur Dataset Evaluasi agar Berulang Tanpa Batas
 
    def count_examples(file_pattern): # Fungsi Pembantu untuk Menghitung Jumlah Contoh dalam File
        file_paths = tf.io.gfile.glob(file_pattern) # Mendapatkan Path File
        count = 0 # Menginisialisasi Penghitung
        for file_path in file_paths: # Iterasi Melalui Setiap File
            dataset = tf.data.TFRecordDataset(file_path, compression_type="GZIP") # Membaca Dataset TFRecord
            count += sum(1 for _ in dataset) # Menambahkan Jumlah Contoh ke Penghitung
        return count # Mengembalikan Total Jumlah Contoh
 
    try: # Blok Try-Except untuk Mengatasi Potensi Error
        train_examples = count_examples(fn_args.train_files) # Menghitung Jumlah Contoh Latihan
        eval_examples = count_examples(fn_args.eval_files) # Menghitung Jumlah Contoh Evaluasi
        
        train_steps = train_examples // 32 # Menghitung Jumlah Langkah Latihan Per Epoch
        eval_steps = eval_examples // 32 # Menghitung Jumlah Langkah Evaluasi Per Epoch
        
        print(f"Training examples: {train_examples}, Steps: {train_steps}") # Mencetak Jumlah Contoh dan Langkah Latihan
        print(f"Eval examples: {eval_examples}, Steps: {eval_steps}") # Mencetak Jumlah Contoh dan Langkah Evaluasi
        
    except Exception as e: # Menangkap Pengecualian Jika Terjadi Error
        print(f"Error counting examples: {e}") # Mencetak Pesan Error
        train_steps = fn_args.train_steps # Menggunakan Langkah Default Jika Perhitungan Gagal
        eval_steps = fn_args.eval_steps # Menggunakan Langkah Default Jika Perhitungan Gagal
 
    tuner = kt.RandomSearch( # Menginisialisasi Tuner dengan Algoritma Random Search
        hypermodel=_build_keras_model, # Menentukan Model Keras yang akan Disesuaikan
        objective="val_accuracy", # Mendefinisikan Tujuan Penyetelan (Memaksimalkan Akurasi Validasi)
        max_trials=3, # Menentukan Jumlah Percobaan Maksimal
        executions_per_trial=1, # Menentukan Jumlah Eksekusi Model per Percobaan
        directory=fn_args.working_dir, # Menentukan Direktori Kerja untuk Tuner
        project_name="weather_tuner" # Menentukan Nama Proyek Tuner
    )
 
    return TunerFnResult( # Mengembalikan Hasil dari tuner_fn
        tuner=tuner, # Mengembalikan Objek Tuner
        fit_kwargs={ # Mengembalikan Argumen untuk Fungsi fit() Model
            "x": train_dataset, # Data Latihan
            "validation_data": eval_dataset, # Data Validasi
            "steps_per_epoch": train_steps, # Langkah-Langkah per Epoch untuk Latihan
            "validation_steps": eval_steps, # Langkah-Langkah per Epoch untuk Validasi
            "epochs": 10 # Jumlah Epoch untuk Latihan
        }
    )

Writing module/weather_tuner_module.py


In [19]:
transform_graph_artifact = transform.outputs['transform_graph'].get()[0] # Mengambil Artefak Grafik Transform
transform_graph_uri = transform_graph_artifact.uri # Mendapatkan URI dari Artefak Grafik Transform
 
print(f"Transform graph URI: {transform_graph_uri}") # Mencetak URI Grafik Transform
 
tuner = Tuner( # Menginisialisasi Komponen Tuner TFX
    tuner_fn='module.weather_tuner_module.tuner_fn', # Menentukan Fungsi Python yang akan Dijalankan oleh Tuner
    examples=transform.outputs['transformed_examples'], # Mengambil Contoh yang Telah Ditransformasi sebagai Input
    transform_graph=transform.outputs['transform_graph'], # Mengambil Grafik Transform sebagai Input
    train_args=tfx.proto.TrainArgs(num_steps=100), # Menentukan Argumen Latihan (Jumlah Langkah untuk Latihan)
    eval_args=tfx.proto.EvalArgs(num_steps=50), # Menentukan Argumen Evaluasi (Jumlah Langkah untuk Evaluasi)
    custom_config={ # Menentukan Konfigurasi Kustom
        'transform_graph_uri': transform_graph_uri # Meneruskan URI Grafik Transform ke Fungsi Tuner
    }
)

Transform graph URI: bravo_alfon\pipeline\Transform\transform_graph\5


In [20]:
context.run(tuner) # Menjalankan Komponen Tuner dalam Konteks Interaktif

Trial 3 Complete [00h 02m 01s]
val_accuracy: 0.8705357313156128

Best val_accuracy So Far: 0.8973214030265808
Total elapsed time: 00h 08m 25s
Results summary
Results in bravo_alfon\pipeline\.temp\6\weather_tuner
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0 summary
Hyperparameters:
conv1_filters: 64
conv2_filters: 96
conv3_filters: 256
dense_units: 192
dropout_rate: 0.3
learning_rate: 0.0001
Score: 0.8973214030265808

Trial 1 summary
Hyperparameters:
conv1_filters: 64
conv2_filters: 128
conv3_filters: 128
dense_units: 64
dropout_rate: 0.3
learning_rate: 0.0001
Score: 0.8705357313156128

Trial 2 summary
Hyperparameters:
conv1_filters: 48
conv2_filters: 64
conv3_filters: 128
dense_units: 128
dropout_rate: 0.4
learning_rate: 0.0001
Score: 0.8705357313156128


ExecutionResult(
    component_id: Tuner
    execution_id: 6
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={})
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}))

## Trainer

**Penjelasan tahapan** : Tahapan ini digunakan untuk melatih model dengan memanfaatkan data yang telah melalui proses transformasi serta menggunakan hasil tuning hyperparameter terbaik yang diperoleh dari tahap sebelumnya. Pada tahap ini, arsitektur model didefinisikan baik dengan konfigurasi default maupun berdasarkan kombinasi hyperparameter yang sudah dioptimalkan. Dataset untuk pelatihan dan evaluasi dibentuk dari file TFRecord hasil transformasi, kemudian dipetakan ke dalam format tensor yang siap dipakai oleh TensorFlow. Proses training dilakukan dengan strategi yang dilengkapi callback seperti TensorBoard untuk logging, EarlyStopping untuk mencegah overfitting, serta callback cetak log per-epoch agar memudahkan pemantauan performa model. Setelah pelatihan selesai, model disimpan dalam format yang siap digunakan pada tahap serving.

In [21]:
def load_best_hyperparameters(tuner_path): # Mendefinisikan Fungsi untuk Memuat Hyperparameter Terbaik dari Hasil Tuner
    try: # Memulai Blok Try-Except untuk Menangani Potensi Kesalahan
        best_hp_file = os.path.join(tuner_path, 'best_hyperparameters.txt') # Mendefinisikan Path Lengkap ke File Hyperparameter
        
        if tf.io.gfile.exists(best_hp_file): # Memeriksa Apakah File Hyperparameter Ada
            with tf.io.gfile.GFile(best_hp_file, 'r') as f: # Membuka File dalam Mode Baca
                best_hps_data = json.load(f) # Memuat Data JSON dari File
            
            print(f"Raw best hyperparameters data: {best_hps_data}") # Mencetak Data Mentah untuk Debugging
            
            if 'values' in best_hps_data: # Memeriksa Apakah Kunci 'values' Ada di Data
                best_hps = best_hps_data['values'] # Mengambil Nilai Hyperparameter dari Kunci 'values'
                print(f"Loaded best hyperparameters: {best_hps}") # Mencetak Hyperparameter yang Berhasil Dimuat
                return best_hps # Mengembalikan Dictionary Hyperparameter
            else: # Jika Kunci 'values' Tidak Ditemukan
                print("No 'values' key found in best hyperparameters data") # Mencetak Pesan Peringatan
                return None # Mengembalikan None
        
        print("No best_hyperparameters.txt found") # Mencetak Pesan Jika File Tidak Ditemukan
        return None # Mengembalikan None
        
    except Exception as e: # Menangkap Kesalahan yang Terjadi
        print(f"Error loading hyperparameters: {e}") # Mencetak Pesan Error
        return None # Mengembalikan None

In [22]:
%%writefile {TRAINER_MODULE_FILE} # Perintah Jupyter untuk Menulis Kode Berikut ke File Python
import tensorflow as tf # Mengimpor TensorFlow
from tfx import v1 as tfx # Mengimpor TFX
from tensorflow_transform import TFTransformOutput # Mengimpor TFTransformOutput untuk Menggunakan Grafik Transformasi
from tensorflow.keras.callbacks import LambdaCallback # Mengimpor LambdaCallback untuk Mencetak Informasi saat Pelatihan
import os # Mengimpor OS untuk Operasi Sistem
import json # Mengimpor JSON untuk Mengurai Data Hyperparameter
 
IMG_SIZE = (128, 128) # Ukuran Gambar yang Digunakan
CLASS_NAMES = ["Cloudy", "Rain", "Shine", "Sunrise"] # Nama-nama Kelas
 
def _build_model(hp_values=None): # Fungsi untuk Membangun Model Keras Berdasarkan Hyperparameter atau Nilai Default
    if hp_values: # Memeriksa Apakah Hyperparameter Disediakan
        print(f"Using tuned hyperparameters: {hp_values}") # Mencetak Pesan Bahwa Hyperparameter Disesuaikan Digunakan
        
        model = tf.keras.Sequential([ # Membuat Model Sequential
            tf.keras.layers.InputLayer(input_shape=(*IMG_SIZE, 3), name="image_xf"), # Layer Input
            
            tf.keras.layers.Conv2D( # Layer Konvolusi Pertama
                filters=hp_values.get('conv1_filters', 32), # Menggunakan Nilai Filter dari Hyperparameter atau Default 32
                kernel_size=(3, 3), activation="relu", padding="same"), # Mengatur Ukuran Kernel, Aktivasi, dan Padding
            tf.keras.layers.MaxPooling2D((2, 2)), # Layer MaxPooling
            
            tf.keras.layers.Conv2D( # Layer Konvolusi Kedua
                filters=hp_values.get('conv2_filters', 64), # Menggunakan Nilai Filter dari Hyperparameter atau Default 64
                kernel_size=(3, 3), activation="relu", padding="same"), # Mengatur Ukuran Kernel, Aktivasi, dan Padding
            tf.keras.layers.MaxPooling2D((2, 2)), # Layer MaxPooling
            
            tf.keras.layers.Conv2D( # Layer Konvolusi Ketiga
                filters=hp_values.get('conv3_filters', 128), # Menggunakan Nilai Filter dari Hyperparameter atau Default 128
                kernel_size=(3, 3), activation="relu", padding="same"), # Mengatur Ukuran Kernel, Aktivasi, dan Padding
            tf.keras.layers.MaxPooling2D((2, 2)), # Layer MaxPooling
            
            tf.keras.layers.Flatten(), # Meratakan Tensor Input
            
            tf.keras.layers.Dense( # Layer Dense
                units=hp_values.get('dense_units', 128), # Menggunakan Jumlah Unit dari Hyperparameter atau Default 128
                activation="relu"), # Fungsi Aktivasi ReLU
            tf.keras.layers.Dropout(hp_values.get('dropout_rate', 0.5)), # Layer Dropout untuk Regularisasi
            
            tf.keras.layers.Dense(len(CLASS_NAMES), activation="softmax"), # Layer Output dengan Jumlah Unit Sama dengan Jumlah Kelas
        ])
        
        model.compile( # Mengompilasi Model
            optimizer=tf.keras.optimizers.Adam( # Menggunakan Optimizer Adam
                learning_rate=hp_values.get('learning_rate', 1e-4) # Menggunakan Learning Rate dari Hyperparameter atau Default 1e-4
            ),
            loss="sparse_categorical_crossentropy", # Mengatur Fungsi Loss
            metrics=["accuracy"] # Mengatur Metrik untuk Evaluasi
        )
        
    else: # Jika Hyperparameter Tidak Disediakan
        print("Using default model architecture") # Mencetak Pesan Bahwa Arsitektur Model Default Digunakan
        
        model = tf.keras.Sequential([ # Membuat Model Sequential dengan Nilai Default
            tf.keras.layers.InputLayer(input_shape=(*IMG_SIZE, 3), name="image_xf"), # Layer Input
            tf.keras.layers.Conv2D(32, (3, 3), activation="relu", padding="same"), # Layer Konvolusi
            tf.keras.layers.MaxPooling2D((2, 2)), # Layer MaxPooling
            tf.keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same"), # Layer Konvolusi
            tf.keras.layers.MaxPooling2D((2, 2)), # Layer MaxPooling
            tf.keras.layers.Conv2D(128, (3, 3), activation="relu", padding="same"), # Layer Konvolusi
            tf.keras.layers.MaxPooling2D((2, 2)), # Layer MaxPooling
            tf.keras.layers.Flatten(), # Meratakan Tensor Input
            tf.keras.layers.Dense(128, activation="relu"), # Layer Dense
            tf.keras.layers.Dropout(0.5), # Layer Dropout
            tf.keras.layers.Dense(len(CLASS_NAMES), activation="softmax"), # Layer Output
        ])
 
        model.compile( # Mengompilasi Model
            optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), # Menggunakan Optimizer Adam dengan Learning Rate Default
            loss="sparse_categorical_crossentropy", # Mengatur Fungsi Loss
            metrics=["accuracy"] # Mengatur Metrik
        )
    
    return model # Mengembalikan Model yang Telah Dikompilasi
 
def run_fn(fn_args: tfx.components.FnArgs): # Fungsi Utama yang Akan Dijalankan oleh Komponen Trainer
    best_hps = None # Menginisialisasi Variabel untuk Hyperparameter Terbaik
    if hasattr(fn_args, 'custom_config') and fn_args.custom_config: # Memeriksa Apakah Konfigurasi Kustom Disediakan
        best_hps = fn_args.custom_config.get('best_hyperparameters') # Mendapatkan Hyperparameter Terbaik dari Konfigurasi Kustom
    
    print(f"Best hyperparameters received: {best_hps}") # Mencetak Hyperparameter yang Diterima
    
    if not best_hps: # Jika Hyperparameter Tidak Ditemukan
        print("No hyperparameters provided, using default model") # Mencetak Pesan Peringatan
        best_hps = {} # Mengatur Variabel Menjadi Dictionary Kosong
    
    tf_transform_output = TFTransformOutput(fn_args.transform_output) # Menggunakan Output Transform untuk Mendapatkan Grafik Transformasi
    feature_spec = tf_transform_output.transformed_feature_spec() # Mendapatkan Spesifikasi Fitur yang Telah Ditransformasi
    
    print("Feature spec:", feature_spec) # Mencetak Spesifikasi Fitur
    
    def create_dataset(file_pattern, batch_size=32, shuffle=False): # Fungsi untuk Membuat tf.data.Dataset
        file_paths = tf.io.gfile.glob(file_pattern) # Mendapatkan Path File yang Sesuai dengan Pola
        print(f"Processing {len(file_paths)} files: {file_paths}") # Mencetak Jumlah dan Daftar File
        
        if not file_paths: # Jika Tidak Ada File yang Ditemukan
            raise ValueError(f"No files found for pattern: {file_pattern}") # Mengangkat Kesalahan
        
        def read_file(filepath): # Fungsi Pembantu untuk Membaca File
            return tf.data.TFRecordDataset(filepath, compression_type="GZIP") # Mengembalikan Dataset dari TFRecord
        
        dataset = tf.data.Dataset.from_tensor_slices(file_paths) # Membuat Dataset dari Daftar Path
        
        dataset = dataset.interleave( # Menginterleave Dataset untuk Pembacaan Paralel
            read_file,
            num_parallel_calls=tf.data.AUTOTUNE,
            cycle_length=min(4, len(file_paths))
        )
        
        def parse_record(serialized_example): # Fungsi Pembantu untuk Menguraikan Rekor
            try: # Memulai Blok Try-Except
                parsed_features = tf.io.parse_single_example(serialized_example, feature_spec) # Menguraikan Contoh
                
                image = parsed_features['image_xf'] # Mendapatkan Tensor Gambar
                label = parsed_features['label_xf'] # Mendapatkan Tensor Label
                
                image = tf.ensure_shape(image, (*IMG_SIZE, 3)) # Memastikan Bentuk (Shape) Tensor Gambar
                label = tf.reshape(label, []) # Mengubah Bentuk Tensor Label menjadi Skalar (Scalar)
                
                return image, label # Mengembalikan Gambar dan Label
                
            except Exception as e: # Menangani Kesalahan Parsing
                print(f"Error parsing record: {e}") # Mencetak Pesan Error
                return tf.zeros((*IMG_SIZE, 3), dtype=tf.float32), tf.constant(0, dtype=tf.int64) # Mengembalikan Data Dummy
        
        dataset = dataset.map(parse_record, num_parallel_calls=tf.data.AUTOTUNE) # Menerapkan Fungsi Parsing ke Dataset
        
        if shuffle: # Jika Perlu Diacak
            dataset = dataset.shuffle(buffer_size=1000) # Mengacak Dataset
            
        dataset = dataset.batch(batch_size) # Mengelompokkan Dataset
        dataset = dataset.prefetch(tf.data.AUTOTUNE) # Melakukan Prefetch untuk Performa
        
        return dataset # Mengembalikan Dataset
    
    try: # Memulai Blok Try-Except untuk Pembuatan Dataset
        train_dataset = create_dataset(fn_args.train_files, batch_size=32, shuffle=True) # Membuat Dataset Latihan
        eval_dataset = create_dataset(fn_args.eval_files, batch_size=32, shuffle=False) # Membuat Dataset Evaluasi
        
        print("Testing dataset...") # Mencetak Pesan
        for images, labels in train_dataset.take(1): # Mengambil Satu Batch untuk Pengujian
            print(f"Batch shapes - Images: {images.shape}, Labels: {labels.shape}") # Mencetak Bentuk Tensor
            break # Keluar dari Loop
            
    except Exception as e: # Menangani Kesalahan Pembuatan Dataset
        print(f"Error creating datasets: {e}") # Mencetak Pesan Error
        print("Using dummy data as fallback...") # Mencetak Pesan Fallback
        dummy_data = tf.data.Dataset.from_tensor_slices(( # Membuat Dataset Dummy
            tf.zeros((100, *IMG_SIZE, 3), dtype=tf.float32),
            tf.zeros((100,), dtype=tf.int64)
        )).batch(32)
        train_dataset = dummy_data # Menggunakan Data Dummy untuk Latihan
        eval_dataset = dummy_data # Menggunakan Data Dummy untuk Evaluasi
    
    print(best_hps) # Mencetak Hyperparameter Terbaik
    model = _build_model(best_hps) # Membangun Model Berdasarkan Hyperparameter Terbaik
    
    print_callback = LambdaCallback( # Membuat Callback untuk Mencetak Log Epoch
        on_epoch_end=lambda epoch, logs: 
            print(f"Epoch {epoch+1}: loss={logs['loss']:.4f}, "
                  f"accuracy={logs['accuracy']:.4f}, "
                  f"val_loss={logs['val_loss']:.4f}, "
                  f"val_accuracy={logs['val_accuracy']:.4f}")
    )
 
    callbacks = [ # Mendefinisikan Daftar Callback
        tf.keras.callbacks.TensorBoard(log_dir=fn_args.model_run_dir), # Callback TensorBoard untuk Logging
        tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5), # Callback EarlyStopping untuk Mencegah Overfitting
        print_callback # Callback Cetak
    ]
    
    history = model.fit( # Melatih Model
        train_dataset,
        validation_data=eval_dataset,
        epochs=100, # Menentukan Jumlah Epoch Maksimal
        callbacks=callbacks # Menambahkan Callback
    )
    
    model.save(fn_args.serving_model_dir, save_format='tf') # Menyimpan Model yang Telah Dilatih

Writing module/weather_trainer_module.py


In [23]:
tuner_path = tuner.outputs['best_hyperparameters'].get()[0].uri # Mendapatkan URI dari Hasil Tuner
print(f"Tuner results path: {tuner_path}") # Mencetak Path Hasil Tuner
 
best_hps = load_best_hyperparameters(tuner_path) # Memuat Hyperparameter Terbaik
print(f"Best hyperparameters: {best_hps}") # Mencetak Hyperparameter Terbaik
 
trainer = Trainer( # Menginisialisasi Komponen Trainer
    run_fn='module.weather_trainer_module.run_fn', # Menentukan Fungsi yang Akan Dijalankan oleh Trainer
    examples=transform.outputs['transformed_examples'], # Mengambil Contoh yang Telah Ditransformasi sebagai Input
    transform_graph=transform.outputs['transform_graph'], # Mengambil Grafik Transformasi sebagai Input
    train_args={'num_steps': 1000}, # Mengatur Jumlah Langkah Latihan
    eval_args={'num_steps': 200}, # Mengatur Jumlah Langkah Evaluasi
    custom_config={ # Menentukan Konfigurasi Kustom
        'best_hyperparameters': best_hps # Meneruskan Hyperparameter Terbaik ke Fungsi run_fn
    }
)

Tuner results path: bravo_alfon\pipeline\Tuner\best_hyperparameters\6
Raw best hyperparameters data: {'space': [{'class_name': 'Int', 'config': {'name': 'conv1_filters', 'default': None, 'conditions': [], 'min_value': 32, 'max_value': 64, 'step': 16, 'sampling': 'linear'}}, {'class_name': 'Int', 'config': {'name': 'conv2_filters', 'default': None, 'conditions': [], 'min_value': 64, 'max_value': 128, 'step': 32, 'sampling': 'linear'}}, {'class_name': 'Int', 'config': {'name': 'conv3_filters', 'default': None, 'conditions': [], 'min_value': 128, 'max_value': 256, 'step': 64, 'sampling': 'linear'}}, {'class_name': 'Int', 'config': {'name': 'dense_units', 'default': None, 'conditions': [], 'min_value': 64, 'max_value': 256, 'step': 64, 'sampling': 'linear'}}, {'class_name': 'Float', 'config': {'name': 'dropout_rate', 'default': 0.3, 'conditions': [], 'min_value': 0.3, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}}, {'class_name': 'Choice', 'config': {'name': 'learning_rate', 'defaul

In [24]:
context.run(trainer) # Menjalankan Komponen Trainer dalam Konteks Interaktif

Best hyperparameters received: {'conv1_filters': 64, 'conv2_filters': 96, 'conv3_filters': 256, 'dense_units': 192, 'dropout_rate': 0.3, 'learning_rate': 0.0001}
Feature spec: {'image_xf': FixedLenFeature(shape=[128, 128, 3], dtype=tf.float32, default_value=None), 'label_xf': FixedLenFeature(shape=[1], dtype=tf.int64, default_value=None)}
Processing 1 files: ['bravo_alfon\\pipeline\\Transform\\transformed_examples\\5\\Split-train\\transformed_examples-00000-of-00001.gz']
Processing 1 files: ['bravo_alfon\\pipeline\\Transform\\transformed_examples\\5\\Split-eval\\transformed_examples-00000-of-00001.gz']


Testing dataset...
Batch shapes - Images: (32, 128, 128, 3), Labels: (32,)
{'conv1_filters': 64, 'conv2_filters': 96, 'conv3_filters': 256, 'dense_units': 192, 'dropout_rate': 0.3, 'learning_rate': 0.0001}
Using tuned hyperparameters: {'conv1_filters': 64, 'conv2_filters': 96, 'conv3_filters': 256, 'dense_units': 192, 'dropout_rate': 0.3, 'learning_rate': 0.0001}
Epoch 1/100
     29/Unknown - 20s 622ms/step - loss: 0.9901 - accuracy: 0.5573Epoch 1: loss=0.9901, accuracy=0.5573, val_loss=0.7744, val_accuracy=0.6504
29/29 [==============================] - 23s 733ms/step - loss: 0.9901 - accuracy: 0.5573 - val_loss: 0.7744 - val_accuracy: 0.6504
Epoch 2/100
29/29 [==============================] - 20s 647ms/step - loss: 0.6082 - accuracy: 0.7542 - val_loss: 0.5696 - val_accuracy: 0.7920
Epoch 3/100
29/29 [==============================] - 20s 656ms/step - loss: 0.4527 - accuracy: 0.8521 - val_loss: 0.5066 - val_accuracy: 0.8009
Epoch 4/100
29/29 [==============================] - 23s 732

INFO:tensorflow:Assets written to: bravo_alfon\pipeline\Trainer\model\7\Format-Serving\assets


INFO:tensorflow:Assets written to: bravo_alfon\pipeline\Trainer\model\7\Format-Serving\assets


ExecutionResult(
    component_id: Trainer
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

## Resolver

**Penjelasan tahapan** : Pada tahap ini, Resolver bertugas mencari model terbaru yang sudah dinyatakan layak pakai (blessed) untuk dipakai di tahap selanjutnya. Dengan strategi LatestBlessedModelStrategy, komponen ini otomatis memilih model paling baru yang lulus evaluasi berdasarkan input model dan model_blessing. Identitas unik diberikan melalui with_id('model_resolver'), dan eksekusi dengan context.run(model_resolver) memastikan pipeline selalu menggunakan model terbaik tanpa perlu pemilihan manual.

In [25]:
model_resolver = Resolver( # Menginisialisasi Komponen Resolver untuk Menyelesaikan Artefak (Misalnya, Mencari Model Terbaru)
    strategy_class=LatestBlessedModelStrategy, # Menggunakan Strategi LatestBlessedModelStrategy untuk Memilih Model Terbaru yang Telah Disetujui (Blessed)
    model=Channel(type=Model), # Menentukan Channel Input untuk Artefak Model
    model_blessing=Channel(type=ModelBlessing) # Menentukan Channel Input untuk Artefak ModelBlessing (Persetujuan Model)
).with_id('model_resolver') # Memberikan ID Unik pada Komponen Resolver

In [26]:
context.run(model_resolver) # Menjalankan Komponen Resolver dalam Konteks Interaktif

ExecutionResult(
    component_id: model_resolver
    execution_id: 8
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

## Evaluator

**Penjelasan tahapan** : Tahap Evaluator berfungsi untuk menilai kualitas model yang sudah dilatih dengan menggunakan TensorFlow Model Analysis (TFMA). Pada tahap ini, dibuat sebuah konfigurasi evaluasi atau eval_config yang berisi pengaturan mengenai spesifikasi model, kunci label yang digunakan, serta metrik-metrik yang akan dihitung seperti jumlah contoh, akurasi kategorikal sparse dengan ambang batas tertentu, presisi, recall, false positives, dan true positives. Data yang digunakan untuk evaluasi diambil dari hasil transformasi khusus pada split eval, sementara model yang dinilai merupakan keluaran dari komponen Trainer. Proses evaluasi ini menghasilkan penilaian objektif mengenai performa model sehingga dapat diketahui apakah model tersebut memenuhi kriteria yang telah ditentukan dan layak untuk dilanjutkan ke tahap berikutnya.

In [27]:
eval_config = tfma.EvalConfig( # Menginisialisasi Objek Konfigurasi Evaluasi
    model_specs=[tfma.ModelSpec(label_key='label_xf')], # Menentukan Spesifikasi Model, 'label_xf' adalah Kunci untuk Label di Data Input
    slicing_specs=[tfma.SlicingSpec()], # Menentukan Spesifikasi Slicing; Slicing Kosong Berarti Evaluasi pada Seluruh Dataset
    metrics_specs=[ # Menentukan Spesifikasi Metrik untuk Evaluasi
        tfma.MetricsSpec(metrics=[ # Daftar Metrik yang akan Dievaluasi
            tfma.MetricConfig(class_name='ExampleCount'), # Menambahkan Metrik untuk Menghitung Jumlah Contoh
            tfma.MetricConfig(class_name='SparseCategoricalAccuracy', # Menambahkan Metrik Akurasi Kategorikal Sparse
                threshold=tfma.MetricThreshold( # Menentukan Ambang Batas untuk Akurasi
                    value_threshold=tfma.GenericValueThreshold( # Ambang Batas Nilai Absolut
                        lower_bound={'value': 0.6} # Menentukan Batas Bawah Akurasi yang Diharapkan
                    ),
                    change_threshold=tfma.GenericChangeThreshold( # Ambang Batas Perubahan Relatif
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER, # Arah Perubahan yang Diinginkan
                        absolute={'value': 0.0001} # Perubahan Minimal yang Dianggap Signifikan
                    )
                )
            ),
            tfma.MetricConfig(class_name='Precision'), # Menambahkan Metrik Presisi
            tfma.MetricConfig(class_name='Recall'), # Menambahkan Metrik Recall
            tfma.MetricConfig(class_name='FalsePositives'), # Menambahkan Metrik False Positives
            tfma.MetricConfig(class_name='TruePositives'), # Menambahkan Metrik True Positives
        ])
    ]
)

In [28]:
evaluator = Evaluator( # Menginisialisasi Komponen Evaluator
    examples=transform.outputs['transformed_examples'], # Mengambil Contoh yang Telah Ditransformasi sebagai Input
    model=trainer.outputs['model'], # Mengambil Model yang Telah Dilatih dari Komponen Trainer
    eval_config=eval_config, # Meneruskan Konfigurasi Evaluasi yang Telah Dibuat
    example_splits=['eval'], # Menentukan Hanya Data 'eval' yang akan Digunakan untuk Evaluasi
    schema=transform.outputs['post_transform_schema'] # Menggunakan Skema Pasca-Transformasi
)

In [29]:
context.run(evaluator) # Menjalankan Komponen Evaluator dalam Konteks Interaktif

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 9
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

In [11]:
eval_result_path = "bravo_alfon/pipeline/Evaluator/evaluation/9" # Mendefinisikan Path ke Hasil Evaluasi dari Komponen Evaluator
 
metrics = tfma.load_metrics(eval_result_path) # Memuat Metrik Evaluasi dari Direktori yang Ditentukan
 
for metric in metrics: # Melakukan Iterasi Melalui Setiap Metrik yang Dimuat
    print(metric) # Mencetak Detail Setiap Metrik Evaluasi

slice_key {
}
metric_keys_and_values {
  key {
    name: "accuracy"
  }
  value {
    double_value {
      value: 0.8938053250312805
    }
  }
}
metric_keys_and_values {
  key {
    name: "loss"
  }
  value {
    double_value {
      value: 0.473164439201355
    }
  }
}
metric_keys_and_values {
  key {
    name: "sparse_categorical_accuracy"
    example_weighted {
    }
  }
  value {
    double_value {
      value: 0.8938053250312805
    }
  }
}
metric_keys_and_values {
  key {
    name: "example_count"
    example_weighted {
    }
  }
  value {
    double_value {
      value: 226.0
    }
  }
}
metric_keys_and_values {
  key {
    name: "precision"
    example_weighted {
    }
  }
  value {
    double_value {
      value: 0.8933333333333333
    }
  }
}
metric_keys_and_values {
  key {
    name: "recall"
    example_weighted {
    }
  }
  value {
    double_value {
      value: 0.8893805309734514
    }
  }
}
metric_keys_and_values {
  key {
    name: "false_positives"
    example_weight

## Pusher

**Penjelasan tahapan** : Tahap Pusher merupakan bagian akhir dari alur TFX yang bertugas untuk push model hasil pelatihan agar dapat digunakan dalam proses deployment. Pada tahap ini, Pusher menerima model yang telah dihasilkan oleh komponen Trainer dan hanya akan melanjutkan proses apabila model tersebut mendapatkan blessing atau persetujuan dari komponen Evaluator yang memastikan kualitasnya sudah sesuai dengan kriteria. Jika model lolos evaluasi, maka Pusher akan menyalin model tersebut ke direktori tujuan yang telah ditentukan dalam konfigurasi push_destination, dalam hal ini sebuah direktori sistem file lokal yang dipersiapkan sebagai lokasi penyimpanan model siap servis. Dengan demikian, model yang sudah melalui validasi menyeluruh akan tersedia di lokasi tersebut dan dapat langsung digunakan untuk kebutuhan serving.

In [30]:
pusher = Pusher( # Menginisialisasi Komponen Pusher
    model=trainer.outputs['model'], # Mengambil Model yang Telah Dilatih dari Komponen Trainer sebagai Input
    model_blessing=evaluator.outputs['blessing'], # Mengambil Artefak 'blessing' dari Komponen Evaluator untuk Memastikan Model Telah Disetujui
    push_destination=pusher_pb2.PushDestination( # Menentukan Tujuan di Mana Model Akan Disimpan
        filesystem=pusher_pb2.PushDestination.Filesystem( # Menggunakan Sistem File Lokal sebagai Tujuan
            base_directory=SERVING_MODEL_DIR # Menentukan Direktori Dasar (Root) untuk Menyimpan Model yang Siap Diservis
        )
    )
)

In [31]:
context.run(pusher) # Menjalankan Komponen Pusher dalam Konteks Interaktif

ExecutionResult(
    component_id: Pusher
    execution_id: 10
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))

## Load Serving Model Architecture

**Penjelasan tahapan** : Menampilkan arsitektur akhir model yang digunakan untuk serving.

In [16]:
saved_model_path = "serving_model/pipeline/1757484741" # Mendefinisikan Path ke Direktori Tempat Model Akhir Disimpan
 
model = tf.keras.models.load_model(saved_model_path) # Memuat Model Keras yang Telah Disimpan dari Direktori
 
model.summary() # Mencetak Ringkasan Struktur Model, Termasuk Lapisan dan Jumlah Parameter

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 128, 128, 64)      1792      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 64, 64, 64)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 64, 64, 96)        55392     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 32, 32, 96)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 32, 32, 256)       221440    
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 16, 16, 256)     